In [1]:
import urllib.request
import zipfile
import os
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [ ]:
train_url = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
urllib.request.urlretrieve(train_url, 'horse-or-human.zip')
with zipfile.ZipFile('horse-or-human.zip', 'r') as zip_ref:
    zip_ref.extractall('data/horse-or-human')

val_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
urllib.request.urlretrieve(val_url, 'validation-horse-or-human.zip')
with zipfile.ZipFile('validation-horse-or-human.zip', 'r') as zip_ref:
    zip_ref.extractall('data/validation-horse-or-human')

In [2]:
TRAINING_DIR = 'data/horse-or-human'
VALIDATION_DIR = 'data/validation-horse-or-human'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import RMSprop

base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False 

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9176\3451537115.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')


In [4]:
model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.0003),
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    verbose=2
)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
33/33 - 24s - 727ms/step - accuracy: 0.8315 - loss: 0.3937 - val_accuracy: 1.0000 - val_loss: 0.0327
Epoch 2/5
33/33 - 15s - 449ms/step - accuracy: 0.9387 - loss: 0.1427 - val_accuracy: 0.9961 - val_loss: 0.0117
Epoch 3/5
33/33 - 15s - 453ms/step - accuracy: 0.9562 - loss: 0.1102 - val_accuracy: 1.0000 - val_loss: 0.0055
Epoch 4/5
33/33 - 15s - 442ms/step - accuracy: 0.9679 - loss: 0.0817 - val_accuracy: 1.0000 - val_loss: 0.0052
Epoch 5/5
33/33 - 15s - 453ms/step - accuracy: 0.9727 - loss: 0.0690 - val_accuracy: 1.0000 - val_loss: 0.0033


In [5]:
model.save("model_05.h5")